Related to user story: [SP11-Item04: General Data Wrapper PoC](https://gitlab.inria.fr/fedbiomed/fedbiomed/-/issues/164)

## Tabular dataset

Workflow of data pre processing:

1. Columns name should be shared with the researcher
2. Data format file to be filled by clinicians.
3. Specify if missing data are allowed for a given columns (Exception). The file will be used for data verification during FL pre-processing,
4. Outlier verification for quantitative data, continuous and discrete, and for dates (Critical warning),
5. Missing data imputation by local mean (or optional NN), or majority voting for discrete labels. Give warnings when missing data are found (for verification a posteriori).
6. Give critical warning when too many missing are found (>50%),
7. Verify that number of available data is greater then minimum required (Error)

Critical warnings have different levels of disclosure to the researcher (1) only the warning, 2) type of warning, 3) type of warning and column affected).

# 1. What is a `data_format_ref` file


A `data_format_ref_file` is a JSON file containing all the skeleton of the data. 
It should be created by a clinician and sent to nodes, in order to check nodes dataset integrity.
In case there is a mismatch between nodes dataset and `data_format_ref_file` 

Structure of a `data_format_file_ref`:



# 2. Creating a `data_format_ref` file using an existing dataset

In this notebook, we provide a way to create `data_fromat_ref_file` using an existing dataset. 
Then, user can edit it (ie change `data_format_file_ref` file, and complete information already contained into the file)

## 2.1 Loading tabular dataset

For that, we provide a `load_tabular_datasets` function able to load every kind of tabular dataset `*.csv`, `.xls`, folder of `*.csv`.

In [1]:
from fedbiomed.common.data_tool.utils import load_tabular_datasets, save_format_file_ref
from fedbiomed.common.data_tool.data_format_ref_cli import get_from_user_multi_view_dataset_fromat_file, edit_format_file_ref

**load_tabular_datasets** method can export several files type (excel file, csv file, and folder containing csv files) 

## Caution! all data are stored in a folder named `/data/` in `fedbiomed/notebooks` 

### 2.1.1 Export Excel File

You can export Excel files as shown in the example below

In [ ]:
!pip install openpyxl #(for opening excel files)
!pip install aenum

In [4]:
# it can load excel files
load_tabular_datasets(r'./data/excel/Exceltest.xlsx')

file found
err 'utf-8' codec can't decode byte 0x8c in position 15: invalid start byte in file ./data/excel/Exceltest.xlsx


{'Exceltest.xlsx':    ID   Age Eligibility
 0    1   45           Y
 1    2   45           Y
 2    3   33           N
 3    4   54           Y
 4    5   45           Y
 5    6   54         NaN
 6    7   34           N
 7    8   54         NaN
 8    9   45         NaN
 9   10   44           Y}

In [5]:
load_tabular_datasets(r'./data/excel/test_excel.xlsx')

file found
err 'utf-8' codec can't decode byte 0x87 in position 11: invalid start byte in file ./data/excel/test_excel.xlsx


{'test_excel.xlsx':      a  b     val
 0    1  a  10.000
 1    2  b   3.000
 2    3  c   3.900
 3    4  d   8.000
 4    5  a   9.009
 5    6  b  11.000
 6    7  c   4.000
 7    8  d   4.900
 8    9  a   9.000
 9   10  b  10.009
 10  11  c  12.000
 11  12  d   5.000
 12  13  a   5.900
 13  14  b  10.000
 14  15  c  11.009
 15  16  d  13.000
 16  17  a   6.000
 17  18  b   6.900
 18  19  c  11.000}

### 2.1.2 Export CSV file


It is a modified version of `pseudo_adni_mod` csv dataset with the first column (`CDRSB.bl`) being converted into a column of string

In [11]:
# simple csv dataset
pseudo_adni = load_tabular_datasets(r'data/pseudo_adni/pseudo_adni_mod_2.csv')
#single_view_dataset = load_tabular_datasets(r'/user/ybouilla/home/Documents/data/pseudo_adni_mod/pseudo_adni_mod.csv')

file found


### 2.1.3 Export a folder containing csv files

for loading a folder (multi view dataset, one has to load the path of the directory (instead of a file)

In [9]:
!tree data/csv_folder

data/csv_folder
├── contatct
├── file1
└── file2

0 directories, 3 files


In [10]:
# folder of dataset
multi_view_dataframe =  load_tabular_datasets('data/csv_folder')


directory found


## 2.2 create the `data_format_ref` file from the loaded dataset

Let's assume the loaded dataset is a reference dataset, and we want to create from this  dataset a `data_format_ref`. 


### workflow logic:

```
for each view:
    for each feature whithin view:
        ask user to select one type within all type present in `Enum` class `DataType` (if user wants to add this feature inside`data_format_ref`)
        ask user to indicate if variable can have missing data or not
        ask user to indicate which data imputation method he wants to select
        (imputation methods are given from the one specified in `Enum` class `ImputationMethods`
        
```

**Default DataType**:

1) KEY 
2) QUANTITATIVE 
3) CATEGORICAL 
4) DATETIME 
5) UNKNOWN (when user doesnot want to specify a variable type)


**Data Imputation Methods**

1) MEAN_IMPUTATION
2) MODE_IMPUTATION
3) KNN_IMPUTATION
4) LINEAR_INTERPOLATION_IMPUTATION

In [ ]:
single_data_format_file_test = get_from_user_multi_view_dataset_fromat_file(pseudo_adni)

In [8]:
single_data_format_file_test

{'global_thresholds': {'min_nb_samples': None, 'min_nb_missing_samples': None},
 'pseudo_adni_mod_2.csv': {'CDRSB.bl': {'data_format': 'DATETIME',
   'data_type': 'DATETIME',
   'values': "<class 'str'>",
   'is_missing_values': False,
   'data_imputation_method': None,
   'data_imputation_parameters': None,
   'data_imputation_variables': None}}}

In [10]:
multi_data_format_file_test = get_from_user_multi_view_dataset_fromat_file(multi_view_dataframe)

+++++++++ Editing Global Thresholds +++++++++++
Do you want to add a threshold for the minimum number of samples each dataset should contain?
1) YES
2) NO
2
Do you want to add a threshold for the minimum number of missing data each feature should contain?
1) YES
2) NO
2
++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++ Now parsing view: file1 +++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++
displaying first 10 values of feature a in view: file1 (n_feature: 1/18)
0    48
1    87
2    46
3    84
4    94
5    18
6    15
7    30
8    54
9    46
Name: a, dtype: int64
number of differents samples: 57 / total of samples: 100
specify data type for a:
1) KEY 
2) QUANTITATIVE 
3) CATEGORICAL 
4) DATETIME 
5) CUSTOM 
6) UNKNOWN 
7) ignore this column
2
data_type DataType.QUANTITATIVE QUANTITATIVE
CATEGORICAL QUANTITATIVE
found  <class 'int'>
found  <class 'numpy.int64'>
int64 <class 'numpy.int64'> [dtype('int64')]
Allow a to have missing values:
1) YES
2) NO
2
displayin

Method imputation selected: KNN_IMPUTATION

please specify k value:
4
imput param {'k': '4'}
get msg []
get msg (selected) []
Please select view that contained desired feature:
1) file1 
2) contatct 
3) file2 
4) select all views
4
end ['file1', 'contatct', 'file2'] False False
get msg ['file1', 'contatct', 'file2']
get msg (selected) ['file1', 'contatct', 'file2']
Please select view that contained desired feature:
1) file1 (selected)
2) contatct (selected)
3) file2 (selected)
4) select all views
5) Finish feature selection
5
Do you want to add more variable to apply to data imputation method ?
1) YES
2) NO
2
displaying first 10 values of feature pkey in view: file1 (n_feature: 18/18)
0    zmixzrgvxrjqxoe sluk
1    vrzahnpfluspdcbfnaqt
2    pnrepvmrxqabdlvisclv
3    gwj luzejwdxzsiljxzd
4    jjdvcnofivbqhirxzdyo
5    e fshtkhnlimpczypnoe
6    qe nlikallf znokwdhk
7    kgenxxkftoeqtrnoteaq
8    abghippigyxzaxtejumu
9    ezfasuuycdda foisjte
Name: pkey, dtype: object
number of differents

KeyboardInterrupt: Interrupted by user

In [4]:
multi_data_format_file_test 

{'global_thresholds': {'min_nb_samples': None, 'min_nb_missing_samples': None},
 'file1': {'a': {'data_format': 'KEY',
   'data_type': 'DATETIME',
   'values': "<class 'numpy.datetime64'>",
   'is_missing_values': False,
   'data_imputation_method': None,
   'data_imputation_parameters': None,
   'data_imputation_variables': None},
  'e': {'data_format': 'KEY',
   'data_type': 'NUMERICAL',
   'values': 'int64',
   'is_missing_values': False,
   'data_imputation_method': None,
   'data_imputation_parameters': None,
   'data_imputation_variables': None},
  'time': {'data_format': 'KEY',
   'data_type': 'DATETIME',
   'values': "<class 'numpy.datetime64'>",
   'is_missing_values': False,
   'data_imputation_method': None,
   'data_imputation_parameters': None,
   'data_imputation_variables': None}}}

Data format file to be filled by clinicians (step 2 int he workflow):

Data format file will be a dictionary specifying the type: 


```
{<view_name>: {<feature_name>: {'data_format' : <data_fomat>,
                                   'data_type': <data_type>,
                                   'type':<values_taken>,
                                   'is_missing_values': <True/False>,
                                   'data_imputation_method': <name_of_dta_imputation_method> ,
                                   'data_imputation_parameters': <parameter_of_iputation_method>,
                                   'lower_bound': <float>,
                                   'upper_bound': <float>,
                                   'categorical_value': <List[values]>}
                                   }
                   }
    }
```

where
* `<view_name>` is the name of the view
* `<feature_name>` is the name of the feature
* `<data_type>` can be categorical or continuous or missing_data or datetime
* `<value_taken>` is the type of the value (eg int, str, float, ...)


Saving `format_file_ref` 

In [9]:
save_format_file_ref(single_data_format_file_test, 'single_format_file_ref_test2')

Data Format Ref File successfully saved at single_format_file_ref_test2


In [8]:

save_format_file_ref(multi_data_format_file_test, 'int_multi_view_format_test')

Format Reference File successfully saved at int_multi_view_format_test


## 2.3 edit   `data_format_ref` file from an existing  `data_format_ref` file

CLI `edit_format_file_ref` allows to specify additional information for checking, including:
 - lower and upper bounds
 - catgorical_value (eg MALE, FEMALE)
 - data_type (not inferred anymore)
 - data imputation method (not inferred anymore)
 - date format (DO NOT WORK YET)

In [11]:
from fedbiomed.common.data_tool.data_format_ref_cli import edit_format_file_ref
from  fedbiomed.common.data_tool import utils

single_data_format_file_test = edit_format_file_ref(single_data_format_file_test)

Now editing format file ref
Edit file: pseudo_adni_mod_2.csv?
1) YES
2) NO
1
Edit variable: CDRSB.bl?
1) YES
2) NO
1
Which field should be modified?
1) data_type
2) Values taken
3) Data Value imputation method
4) Cancel Operation
2
action 2 4 {'1': <function ask_for_data_type at 0x7fc559190a60>, '2': <function ask_for_categorical_values at 0x7fc559190b80>, '3': <function ask_for_data_imputation_method at 0x7fc559190af0>, '4': <function cancel_operation at 0x7fc559190820>}
enter possible values (separated by ",")a,b,c,d,e,f,g,h,i,j,k
Continue Editing variable: CDRSB.bl?
1) YES
2) NO
2
Edit variable: ADAS11.bl?
1) YES
2) NO
1
Which field should be modified?
1) data_type
2) lower bound
3)upper bound
4) Data Value imputation method
5) Cancel Operation
2
action 2 5 {'1': <function ask_for_data_type at 0x7fc559190a60>, '2': <function ask_for_lower_bound at 0x7fc5591908b0>, '3': <function ask_for_upper_bound at 0x7fc559190940>, '4': <function ask_for_data_imputation_method at 0x7fc559190af0>,

In [13]:
save_format_file_ref(single_data_format_file_test, 'single_format_file_ref_test3')

Data Format Ref File successfully saved at single_format_file_ref_test3


In [1]:
from fedbiomed.common.data_tool.data_format_ref_cli import get_from_user_multi_view_dataset_fromat_file, edit_format_file_ref
from  fedbiomed.common.data_tool import utils

multi_data_format_file_test = utils.load_format_file_ref('int_multi_view_format_test')

edit_multi_view_format_file_test = edit_format_file_ref(multi_data_format_file_test)

+++++++ Now editing format file ref ++++++++++
Edit file: file1?
1) YES
2) NO
1
Edit variable: a?
1) YES
2) NO
2
Edit variable: e?
1) YES
2) NO
2
Edit variable: i?
1) YES
2) NO
1
Which field should be modified?
1) data_type
2) Data Value imputation method
3) Cancel Operation
1
action 1 3 {'1': <function ask_for_data_type at 0x7f316c3cfd30>, '2': <function ask_for_data_imputation_method at 0x7f316c3cfdc0>, '3': <function cancel_operation at 0x7f316c3cfaf0>}
1) KEY 
2) QUANTITATIVE 
3) CATEGORICAL 
4) DATETIME 
5) CUSTOM 
6) UNKNOWN 
7) cancel operation
6
Continue Editing variable: i?
1) YES
2) NO
2
Edit variable: 0?
1) YES
2) NO
1
Which field should be modified?
1) data_type
2) Values taken
3) Data Value imputation method
4) Cancel Operation
2
action 2 4 {'1': <function ask_for_data_type at 0x7f316c3cfd30>, '2': <function ask_for_categorical_values at 0x7f316c3cfe50>, '3': <function ask_for_data_imputation_method at 0x7f316c3cfdc0>, '4': <function cancel_operation at 0x7f316c3cfaf0>}
en

In [2]:
save_format_file_ref(edit_multi_view_format_file_test, 'edit_multi_view_format_file_test3')

Format Reference File successfully saved at edit_multi_view_format_file_test3


In [4]:
edit_multi_view_format_file_test

{'global_thresholds': {'min_nb_samples': 40, 'min_nb_missing_samples': 20},
 'file1': {'a': {'data_format': 'QUANTITATIVE',
   'data_type': 'DISCRETE',
   'values': 'int64',
   'is_missing_values': True,
   'data_imputation_method': 'LINEAR_INTERPOLATION_IMPUTATION',
   'data_imputation_parameters': None,
   'data_imputation_variables': {'file1': ['e',
     'i',
     'o',
     '0',
     '1',
     '2',
     '3',
     'time',
     'pressure',
     'sp02',
     'a.1',
     'e.1',
     'i.1',
     'o.1',
     'gender',
     'blood type',
     'pkey'],
    'contatct': [],
    'file2': []}},
  'e': {'data_format': 'QUANTITATIVE',
   'data_type': 'DISCRETE',
   'values': 'int64',
   'is_missing_values': False,
   'data_imputation_method': None,
   'data_imputation_parameters': None,
   'data_imputation_variables': None},
  'i': {'data_format': 'UNKNOWN',
   'data_type': 'UNKNOWN',
   'values': ['U', 'N', 'K', 'N', 'O', 'W', 'N'],
   'is_missing_values': False,
   'data_imputation_method': Non

## 2.4 create custom DataType

This may be re work because depends on a third party package `aenum`